In [2]:
from pyspark import SparkConf, SparkContext
import os
import time

In [3]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("random_walk")
sc = SparkContext(conf=conf)

In [4]:
def mapper1(x):
    return_list = []
    for element in x[1][0][1]:
        return_list.append((int(element), 0.8 * ( x[1][1] / x[1][0][0])))
    return return_list
def mapper3(x):
    if x[1][1] == None:
        return (x[0], x[1][0])
    return (x[0], x[1][0] + x[1][1])
    
    

        

In [5]:
data = sc.textFile("p2p-Gnutella04.txt")
nodes = 10876

extractor1 = data.filter(lambda x: x[0] != '#').map(lambda x : (int(x.split('\t')[0]), int(x.split('\t')[1])))
extractor2 = data.filter(lambda x: x[0] != '#').map(lambda x : (int(x.split('\t')[0]), 1))

r_new = extractor1.flatMap(lambda x: [(x[0],0.2 * 1/nodes), (x[1],0.2 * 1/nodes)]).reduceByKey(lambda x,y: x)
ans = extractor1.flatMap(lambda x: [(x[0],0.2 * 1/nodes), (x[1],1/nodes)]).reduceByKey(lambda x,y: x)

table = extractor1.groupByKey().mapValues(tuple)
Count = extractor2
degree = Count.reduceByKey(lambda x,y: x+y)
M = degree.join(table)


for i in range(20):
    R_old = ans
    R_new = r_new
    tmp = M.join(R_old) #(j, (out_deg, (i....), r_old))
    to_sum_M = tmp.flatMap(mapper1).reduceByKey(lambda x,y: x+y)
    af_mul = R_new.leftOuterJoin(to_sum_M).map(mapper3)
    y = af_mul.map(lambda x : x[1]).sum()
    ans = af_mul.map(lambda x: (x[0], x[1] + (1-y)/nodes ))
    
Sorted = ans.sortBy(lambda x: x[1], False).collect() 

for i in range(10):
    print('%d\t%f'%(Sorted[i][0],round(Sorted[i][1], 6)))










1056	0.000632
1054	0.000629
1536	0.000524
171	0.000512
453	0.000496
407	0.000485
263	0.000480
4664	0.000470
261	0.000463
410	0.000462


# RageRank




## preprocess:


```python=
def mapper1(x):
    return_list = []
    for element in x[1][0][1]:
        return_list.append((int(element), 0.8 * ( x[1][1] / x[1][0][0])))
    return return_list

def mapper3(x):
    if x[1][1] == None:
        return (x[0], x[1][0])
    return (x[0], x[1][0] + x[1][1])
```


首先會先做出兩組pair的RDD，

```python=
extractor1 = data.filter(lambda x: x[0] != '#').map(lambda x : (int(x.split('\t')[0]), int(x.split('\t')[1])))
extractor2 = data.filter(lambda x: x[0] != '#').map(lambda x : (int(x.split('\t')[0]), 1))

r_new = extractor1.flatMap(lambda x: [(x[0],0.2 * 1/nodes), (x[1],0.2 * 1/nodes)]).reduceByKey(lambda x,y: x)
r_old = extractor1.flatMap(lambda x: [(x[0],0.2 * 1/nodes), (x[1],1/nodes)]).reduceByKey(lambda x,

```

分別為r_new以及r_old:


$$ r_{new} =  (i ,\dfrac{1-\displaystyle \beta}{N}) $$
$$ r_{old} =  (i ,\dfrac{1}{N}) $$

N, beta分別為node的數量與定義好的常數

做法是把input file做一次flatmap給予pair正確的數值，把edge起點和終點的node以(node_id, initial_value)的方式都丟進RDD，這麼一來就會把每個node都創造出來且不會遺漏，然後再做一次reduce把key一樣的放在起並只return其中一個，這樣就可以避免node重複出現。

接者做出matrix:

```python=
table = extractor1.groupByKey().mapValues(tuple)
Count = extractor2
degree = Count.reduceByKey(lambda x,y: x+y)
M = degree.join(table)
tmp = M.join(R_old)
```


$$tmp =  (j, (outdeg_j, (node1_{j->i}, node2_{j->i}, node3_{j->i}, node4_{j->i}, ...), R_{old, j}) $$


先將j所有的pair從extractor group在一起，形成(j, node1,node2...)。
接者，將原始data map成(j, 1)放進Count裡面，然後做reduce，把1全部加起來，找出node j的out degree(j, outdeg)。最後把兩個RDD join在一起，把剛剛做出來的RDD(j, (outdeg ,(node1, node2, node3...)))和R_old join起來。

## iteration part:

```python=
for i in range(20):
    R_old = ans
    R_new = r_new
    tmp = M.join(R_old) #(j, (out_deg, (i....), r_old))
    to_sum_M = tmp.flatMap(mapper1).reduceByKey(lambda x,y: x+y)
    af_mul = R_new.leftOuterJoin(to_sum_M).map(mapper3)
    y = af_mul.map(lambda x : x[1]).sum()
    ans = af_mul.map(lambda x: (x[0], x[1] + (1-y)/nodes ))
```

將r_new初始化為：

$$ r_{new} =  (i ,\dfrac{1-\displaystyle \beta}{N}) $$

r_old初始為上一次iteration的r_new

產生一個RDD:

$$ (i, r_{tmp}) $$


做法是把剛剛做完的matrix RDD (tmp) 用mapper1 map成：
$$(nodek_{j->i}, 0.8*\frac{r_{old,j}}{outdeg_j})$$

接者把這些pair和r_new join起來，並且用mapper3把pair裡面的value相加。下一步，會算出r_new的和，並且對r_new做一次map，normalize成：

$$ (node, r_{new} + \frac {1-summation_{r_{new}}}{N})$$

完成一次iteration



結果：
```
1056	0.000632
1054	0.000629
1536	0.000524
171	0.000512
453	0.000496
407	0.000485
263	0.000480
4664	0.000470
261	0.000463
410	0.000462
```